In [1]:
from wsgiref.simple_server import make_server
import falcon

# Falcon follows the REST architectural style, meaning (among
# other things) that you think in terms of resources and state
# transitions, which map to HTTP verbs.
class ClassifyResourceV1:
    def on_get(self, req, resp):
        # Gets the img query parameter
        img_url = req.get_param('img')
        # Creates a result dictionary with a single entry
        result = dict(src_img=img_url)
        # Sets the response content to the result dictionary
        resp.media = result
        """Handles GET requests"""
        resp.status = falcon.HTTP_200  # This is the default status


# falcon.App instances are callable WSGI apps
# in larger applications the app is created in a separate file
app_v1 = falcon.App()

# Resources are represented by long-lived class instances
classify_v1 = ClassifyResourceV1()

# things will handle all requests to the '/classify' URL path
app_v1.add_route('/classify', classify_v1)

In [2]:
def run_server(app):
    with make_server('', 9999, app) as httpd:
        print('Serving on port 9999...')

        # Serve until process is killed
        try:
            httpd.serve_forever()
        except KeyboardInterrupt:
            print('Stopping web server')


In [3]:
run_server(app_v1)

Serving on port 9999...


127.0.0.1 - - [19/Nov/2022 19:25:58] "GET /classify HTTP/1.1" 200 17
127.0.0.1 - - [19/Nov/2022 19:25:59] "GET /favicon.ico HTTP/1.1" 404 26
127.0.0.1 - - [19/Nov/2022 19:26:27] "GET /classify?img=hello HTTP/1.1" 200 20


Stopping web server


KeyboardInterrupt: 

In [ ]:
# Assignment 12.2 Implement a function that makes image classification predictions on an image URL.

from pathlib import Path
import tempfile
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions


import numpy as np
import requests

import json

# Downloads and loads the RestNetModel
model = ResNet50(weights='imagenet')

In [ ]:
def load_img_url(img_url, target_size):
    print('Loading image')
    r = requests.get(img_url)
    if r.status_code != 200:
        print('Could not download image')
        return None

    img_suffix = '.{}'.format(img_url.split('.')[-1])

    with tempfile.NamedTemporaryFile(suffix=img_suffix) as fp:
        fp.write(r.content)
        tf_path = Path(fp.name)
        img = image.load_img(tf_path, target_size=target_size)

    return img

In [ ]:
def make_prediction(model, img_url, target_size=(224, 224), top=5):
    print('Making prediction')
    result = dict(src_img=img_url)
    img = load_img_url(img_url, target_size)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    preds = model.predict(x)

    result['predictions'] = []

    for class_name, class_description, score in decode_predictions(preds, top=top)[0]:
        result['predictions'].append(dict(
            class_name=str(class_name),
            class_description=str(class_description),
            score=float(score)
        ))

    return result

In [ ]:
# TODO: Test the `make_prediction` function using img_urls

# TODO: Add img_urls for testing
img_urls = ['https://stardewvalleywiki.com/mediawiki/images/9/93/Greenhouse.png']

for img_url in img_urls:
    print('Pulling images from urls')
    result = make_prediction(model, img_url)
    print('-----------------------------')
    print('')
    # Prints a pretty version of the result
    print(json.dumps(result, indent=2))
    print('-----------------------------')
    print()

In [ ]:
class ClassifyResourceV2:
    def on_get(self, req, resp):
        # Gets the img query parameter
        img_url = req.get_param('img')
        # make prediction
        result = make_prediction(model, img_url)
        # Sets the response content to the result dictionary
        resp.media = result

        resp.status = falcon.HTTP_200  # This is the default status

app_v2 = falcon.App()

classify_v2 = ClassifyResourceV2()

app_v2.add_route('/classify', classify_v2)

In [ ]:
run_server(app_v2)